In [ ]:
import numpy as np
import pandas as pd
from glob import glob
from scipy.signal import periodogram, medfilt2d
from scipy.interpolate import interp1d
from scipy.fftpack import next_fast_len

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib notebook
figsize=[19 , 5]
isPlot = False

In [ ]:
maxFreqSearch = 60
maxHarmonic = 50
peddingFactor = 100
detectionThresh = 2
medFiltWindHz = 50
sFactor = 1.1

In [ ]:
def HPS(x, t, 
        maxFreqSearch,
        maxHarmonic,
        peddingFactor,
        detectionThresh,
        medFiltWindHz,
        sFactor,
        isPlot=False):
    # upsampled spectrum for later HPS with better resolution:
    nfft = next_fast_len(x.shape[0]) * peddingFactor
    f, pxx = periodogram(x, fs=1 / t[1], return_onesided=True, axis=0, nfft=nfft)

    # white signal approximation:
    condPre = f < maxFreqSearch * maxHarmonic * sFactor
    pxx = pxx[condPre]
    f = f[condPre]
    fWhite = f[::peddingFactor]
    pxxDec = pxx[::peddingFactor]
    winSize = (np.round(medFiltWindHz/ fWhite[1] / 2) * 2).astype(int) + 1
    pxxMed = medfilt2d(np.log(pxxDec), (winSize, 1))
    interpFuns = [interp1d(fWhite, pxxMed[:, channel]) for channel in range(pxxMed.shape[1])]
    cond = f < maxFreqSearch * maxHarmonic 
    pxx = pxx[cond]
    f = f[cond]
    pxxMed = np.concatenate([np.atleast_2d(interpFun(f)) for interpFun in interpFuns], axis=0).T

    # pre-whitening
    logPxxDetrand = np.log(pxx) - pxxMed

    # thresholding
    logPxxDetrand[logPxxDetrand < detectionThresh] = 0
    logPxxDetrand += 1
    # channel product
    probAxisProd = np.prod(logPxxDetrand, axis=1)

    # Harmonic Product spectrum implementation:
    nSearch = (f < maxFreqSearch).sum()
    probHarmonicProd = np.ones((nSearch, ))
    maxLoc = [] #monitoring variable
    snr = [] #monitoring variable
    for harmonic in range(1, maxHarmonic + 1):
        probHarmonicProd *= probAxisProd[:nSearch * harmonic:harmonic]

        # monitoring and visualization:
        maxLoc.append(np.argmax(probHarmonicProd))
        snr.append(probHarmonicProd[maxLoc[-1]] / probHarmonicProd.mean())
        if isPlot:
            plt.figure(figsize=figsize)
            plt.title("Harmonic No. {harmonic}".format(harmonic=harmonic))
            plt.semilogy(f[:nSearch], probHarmonicProd)
            plt.semilogy(f[:nSearch], probHarmonicProd / probAxisProd[:nSearch * harmonic:harmonic])
            plt.legend(['after', 'before'])
            plt.grid()
            plt.show()
    maxFreq = f[maxLoc]
    return snr, maxFreq

In [ ]:
files = pd.Series(glob('*.csv'))
files = pd.concat([files, files.str.split('-').apply(lambda x:''.join([a for a in x if 'bearing' not in a]))], axis=1)
for s in files.iloc[:, 1].unique():
    subGroup = files.loc[files.iloc[:, 1] == s, 0].values
    print(subGroup)
    print('---')
    
    df = pd.concat([pd.read_csv(file) for file in subGroup], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    t = df.timestamp.values
    x = df.iloc[:, 1:]
    snr, maxFreq = HPS(x, t, maxFreqSearch, maxHarmonic, peddingFactor, detectionThresh, medFiltWindHz, sFactor, 
                   isPlot=False)
    
    plt.figure(figsize=figsize)
    plt.plot(maxFreq)
    plt.ylabel('$Max Frequency [Hz]')
    plt.xlabel('Harmonic')
    plt.ylim([maxFreq[-1] - 0.025, maxFreq[-1] + 0.025])
    plt.grid()
    plt.show()
    
    plt.figure(figsize=figsize)
    plt.plot(maxFreq)
    plt.ylabel('$Max Frequency [Hz]')
    plt.xlabel('Harmonic')
    plt.grid()
    plt.show()

    plt.figure(figsize=figsize)
    plt.plot(snr)
    plt.xlabel('Harmonic')
    plt.grid()
    plt.ylabel('frac{Max Peak}{Mean value}')
    plt.show()